Задание
К вам на анализ попал набор данных  о работе сервиса по продаже авиабилетов с дневной агрегацией. Вашей задаче будет рассчитать определенные показатели для каждого месяца, представленные ниже. 

Данные содержат:

Дату

Идентификатор сессии

Идентификатор пользователя

Сумму оплаты во время сессии

Оценку сервиса (она запрашивается у пользователя после оформления билетов)

 

Используя данные, рассчитайте показатели для каждого месяца:

Количество сессий

MAU

WAU (среднее)

DAU (среднее)

Липкость 

Конверсию в продажу

ARPU

ARPPU

Удовлетворенность клиентов

Для второй недели января рассчитайте Retention с 1 по 10 недели. Постройте график, сделайте выводы.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [279]:
# Загрузка данных
df = pd.read_csv('data/data.csv')

In [ ]:
df 

In [ ]:
#Проверяю типы данных и пропуски
df.info()

In [ ]:
#Статистическая информация
df.describe(include = 'all')

In [281]:
#Переименовываю столбецы 
df.rename(columns 
                 = {'Дата': 'date', 'ID сессии': 'id_ses', 'Оценка': 'evaluation', 
                    'ID клиента': 'id_user', 'Доход': 'income'}, inplace = True)
#Меняю тип данных в колонках с датой
df.date = df.date.astype("datetime64[ns]")

In [ ]:
print(f'В датасете {len(df) - len(df.drop_duplicates(subset = "id_ses"))} дубликатов строк')

In [282]:
#Удаляю дубликаты
#df.drop_duplicates(inplace = True)
df.drop_duplicates(subset = "id_ses", inplace = True)
df.reset_index(drop = True, inplace = True)

In [ ]:
# Проверка на дубликаты
print(f'В датасете {len(df) - len(df.drop_duplicates())} дубликатов строк')

In [283]:
# добавляю столбец month
df['month'] = df.date.dt.month
df['day'] = df.date.dt.day
df['week'] = df.date.dt.to_period('W')

In [ ]:
df

## Расчет показателей

In [284]:
df_session = df.groupby(by = 'month').id_ses.count().reset_index()
df_session.rename(columns = {'id_ses': 'quan_ses'}, inplace = True)
df_mau = df.groupby(by = 'month').id_user.nunique().reset_index()
df_mau.rename(columns = {'id_user': 'mau'}, inplace = True)
df_dau = df.groupby(by = 'date').id_user.nunique().reset_index()
df_dau.rename(columns = {'id_user': 'dau'}, inplace = True)
df_wau = df.groupby(by = 'week').id_user.nunique().reset_index()
df_wau.rename(columns = {'id_user': 'wau'}, inplace = True)
df_conv = df.query("income > 0").groupby(by = 'month').income.count().reset_index()
df_conv.rename(columns = {'income': 'conv'}, inplace = True)
df_purchase = df.query("income > 0").groupby(by = 'month').id_user.nunique().reset_index()
df_purchase.rename(columns = {'id_user': 'purchase'}, inplace = True)
df_income = df.groupby(by = 'month').income.sum().reset_index()
df_income.rename(columns = {'income': 'income_month'}, inplace = True)


In [285]:
df = df.merge(df_dau, how='left', on='date')
df = df.merge(df_wau, how='left', on='week')
df = df.merge(df_mau, how='left', on='month')
df = df.merge(df_session, how='left', on='month')
df = df.merge(df_conv, how='left', on='month')
df = df.merge(df_purchase, how='left', on='month')
df = df.merge(df_income, how='left', on='month')


In [268]:
dau_mean = df.groupby(by = 'month').dau.mean().reset_index()
dau_mean.rename(columns = {'dau': 'dau_mean'}, inplace = True)
wau_mean = df.groupby(by = 'month').wau.mean().reset_index()
wau_mean.rename(columns = {'wau': 'wau_mean'}, inplace = True)
df = df.merge(dau_mean, how='left', on='month')
df = df.merge(wau_mean, how='left', on='month')


In [276]:
df.head(3)

,date,id_ses,id_user,income,evaluation,month,day,week,dau,wau,mau,quan_ses,conv,purchase,income_month,dau_mean,wau_mean
0,2021-01-05,eeehjltlceclayhacecerhyyxhrxs,45870310,0.0,NaN,1,5,2021-01-04/2021-01-10,1,8,2495,2823,1333,1263,2.227633e+07,162.910025,872.0418
1,2021-01-06,eeehrcecacecerlrhacecerygejeb,76545402,0.0,NaN,1,6,2021-01-04/2021-01-10,2,8,2495,2823,1333,1263,2.227633e+07,162.910025,872.0418
2,2021-01-06,eeehalhalhjaracecerxhrhtrm,424242,0.0,NaN,1,6,2021-01-04/2021-01-10,2,8,2495,2823,1333,1263,2.227633e+07,162.910025,872.0418


In [286]:
df.groupby(by = 'month', as_index = False)[['dau' , 'wau', 'mau',  'conv', 'quan_ses', 'purchase', 'income_month']].mean()


,month,dau,wau,mau,conv,quan_ses,purchase,income_month
0,1,162.910025,872.041800,2495.0,1333.0,2823.0,1263.0,2.227633e+07
1,2,343.583212,2022.900236,7546.0,1233.0,8911.0,1189.0,2.364104e+07
2,3,353.822075,1923.008076,3610.0,917.0,4086.0,887.0,1.536327e+07


In [ ]:
#Количество сессий в месяц
df_session = df.groupby(by = 'month').id_ses.count().sort_index()
#MAU
df_mau = df.groupby(by = 'month').id_user.nunique()
dau = df.groupby(by = 'date').id_user.nunique().reset_index()
wau = df.groupby(by = 'week').id_user.nunique().reset_index()

dau_ = df.groupby(['month', 'day']).agg({'id_user': 'nunique'}).mean()
df_dau = pd.DataFrame(df.groupby(df['date'])['id_user'].nunique())
dau_mean = df_dau.groupby(df['date'].dt.month)['id_user'].mean()
df_wau = df.groupby(df['date'].dt.to_period('W'))['id_user'].nunique()
#df_dau = df[[df.groupby(by = 'date')]].groupby(by = 'month')
#df_dau = df.groupby([ 'date','month']).id_user.nunique().agg('mean')
#df_arpu = 
print(f'Количество сессий в Январе - {df_session.loc[1]}, в Феврале - {df_session.loc[2]}, \
      в Марте - {df_session.loc[3]}')
print(f'MAU в Январе - {df_mau.loc[1]}, в Феврале - {df_mau.loc[2]}, \
      в Марте - {df_mau.loc[3]}')


In [ ]:
df_temp = df.merge(dau, how='left', on='date')

In [ ]:
df_temp

In [ ]:
df.groupby(['month', 'day']).agg({'id_user': ['nunique']}).reset_index()





In [ ]:
df_dau.groupby(pd.TimeGrouper(freq='M')).mean()

In [ ]:
df_wau

In [ ]:
dau_df = pd.DataFrame(df_dau).reset_index()
dau_df.columns = ['date', 'dau']

In [ ]:
dau_df.sample(10)

In [ ]:
s = dau_df.groupby(df.date.dt.month)dau'].mean()

In [ ]:
s